# Studying guided modes with PyMoosh

Modes are solutions of Maxwell's equations in a multilayered structure absent any source or any outside excitation. In order to find them, one has theoretically to solve the dispersion relation for a given $\omega$, and by doing so find the wavevector $k_x$ in the complex plane, and then to find the corresponding coefficients of the plane waves in each layer to represent the mode. The mode can be interpreted physically as an excitation which took place inside the structure at $\pm \infty$, and thus as a guided or a leaky mode. 

In the following, we will
1. Explain what modes are, exactly, and how we should derive them
2. Show what function to use to find guided modes, and how to represent them
3. Present a way to study the dispersion relation of these guided modes

## Dielectric waveguide

We first study the dielectric waveguide, constituted by a 600 nm thick layer of glass typically (with a refractive index of 1.5) surrounded with air. 

In [1]:
from PyMoosh import *
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150


structure = Structure([1.,2.25],[0,1,0],[0,600,200])

List of materials:
Simple, non dispersive: epsilon= 1.0
Simple, non dispersive: epsilon= 2.25


The wavelength in vaccum (which actually defines the frequency $\omega$) is taken to be $\lambda = 600$ nm. The effective index of a guided mode is defined by $$ n_\text{eff} = \frac{k_x}{k_0}$$ where $k_0 = \frac{2\pi}{\lambda} = \frac{\omega}{c}$. We take first a random effective index and find the value of the function `dispersion`

In [2]:
wavelength=600.
k_0 = 2*np.pi/wavelength
k_x = (1.0718280558+0.00000000001j)*k_0
f=dispersion(k_x,structure,wavelength,0)
print(k_x,f)

(0.011224157153375701+1.0471975511965976e-13j) 1.2585506192481664


In PyMoosh, we use a trick instead of solving the dispersion relation. We use the reflection coefficient and try to find the zeros of its inverse. Modes are actually poles of the reflection coefficient, so that finding the zeros of the inverse of the modulus of the reflection coefficient is enough. Since these zeros are minima of $\frac{1}{|r|}$, we can use a steepest descent (not the most effective way to find the zeros, probably) and we have a halting criterium (when $\frac{1}{|r|}$ is small enough). 

Remark: The determination of the square root in the outside medium has to be changed though, otherwise poles of the reflection coefficient can be hidden under another Riemann sheet.

In order to use PyMoosh's steepest descent, you have to know where to start in the complex plane (by giving the effective index, which can be complex), provide 



In [3]:
solution2 = steepest(1.05,1e-10,1000,structure,wavelength,0.)

In [4]:
#print(solution,dispersion(solution*k_0,structure,wavelength,0.))
print(solution2,dispersion(solution2*k_0,structure,wavelength,0.))

(1.0393548947630198-6.635754054764027e-15j) 2.5403703678492868e-12


### Mapping the reflection coefficient in the complex plane

In [5]:
X,Y,T = complex_map(structure,wavelength,0.,[1.,1.6],[-0.3,0.3],200,200)

NameError: name 'complex_map' is not defined

In [ ]:
    plt.contourf(X,Y,np.sqrt(np.real(T)))
    plt.grid(True)
    plt.show()

### Finding the modes of the structure

Now there is an automated way to find the different modes in this structure without having to look at the map or launch by hand a few steepest descents to find their effective indexes. The function `Guided_modes` does just that, and gives all the different effectives indexes that satisfy the halting condition of the steepest descent.

In [ ]:
modes = Guided_modes(structure,wavelength,0,1.01,1.6)
print(modes)

Once the modes have been found, it is possible to access their profile. You have to specify the effective 

In [ ]:
    # Let first modify the thickness of the superstrat which is shown
    structure.thickness[0]=200
    x,prof = Profile(structure,modes[1],wavelength,0)
    plt.plot(x,np.real(prof),linewidth = 2)


In [ ]:
for k in range(len(modes)):
    x,prof = Profile(structure,modes[k],wavelength,0)
    plt.plot(x,np.abs(prof)**2,linewidth = 2)
plt.ylabel('Mode profile (a.u), intensity')
plt.show()


## Coupled dielectric waveguides

Now, we will study quickly two coupled dielectric waaveguides, just for fun, because the modes are nice. First, as is obvious from the values of the effective index, the modes go in pair (a symetric and an antisymetric mode). Here, we had to increase the number of starting points for the steepest descent to 80, in order not to miss any mode.


In [ ]:
coupled = Structure([1.,2.25],[0,1,0,1,0],[200,1200,300,1200,300])
modes = Guided_modes(coupled,wavelength,0,1.00,1.6,initial_points = 80)
for k_x in modes:
    print(k_x)

Now that we are sure we have all the modes, let's represent them :

In [ ]:
for k in range(len(modes)):
    x,prof = Profile(coupled,modes[k],wavelength,0)
    plt.plot(x,np.real(prof),linewidth = 1)
plt.ylabel('Mode profile (a.u)')
plt.xlabel('Position (nm)')
plt.show()


## Surface plasmon

An emblematic guided mode is the "surface plasmon" or sometimes "surface plasmon polariton". We will keep the "surface plasmon" denomination, because this kind of surface wave propagating at the surface of a metal is not a plasmon, strictly speaking and even less a polariton. The problem for PyMoosh is that it assumes that there is more than just one interface in the structure. You thus have to invent one, in the middle of nowhere. Here, we add one in the silver after 0 nm. Not that we are in TM polarization here.

In [ ]:
metallic_interface = Structure([1.,'Gold'],[0,1,1],[1000,0,100])
wavelength = 600
modes = Guided_modes(metallic_interface,wavelength,1,1.000,1.3)
print(modes)

In [ ]:
x,prof = Profile(metallic_interface,modes[0],wavelength,1)
plt.plot(x,np.abs(prof)**2,linewidth = 2)
plt.ylabel('Mode profile (a.u), intensity')
plt.xlabel('Position (nm)')
plt.show()


As you can see, the effective index of the surface plasmon is not very high. Which is a pity, because this means there will not be much miniaturization to expect from surface plasmon based cavities. We can try with a gap-plasmon...

## Gap-plasmon

More challenging, we will now try to find the mode (it's unique) which is able to propagate in a nanometric space between two metallic regions. In this regime, the mode propagates almost more inside the metal than in the dielectric. As a consequence, it is slowed down by the metal (the Poynting vector is negative inside the metal and the mean Poynting vector is in fact linked to the group velocity). While there is air in the gap, the effective index of the mode can go much higher than in any dielectric material. 

In [ ]:
gap = Structure([1.,'Gold'],[1,0,1],[50,3,50])
wavelength = 600
gap_plasmon = steepest(4,1e-12,10000,gap,wavelength,1.)
print(gap_plasmon)

The effective index is as high as 8, with a massive imaginary part, for a 3 nm wide gap. Which is not much. `Guided_modes` will fail here because its limited number of steps before giving up does not allow it to reach the effective index of the mode. Hence the direct use of the steepest descent with adapted parameters (and a maximum number  of step of 10000). In order to represent the mode correctly, one has to adapt the size of a pixel and take it down to 0.1 nm.

In [ ]:
x,prof = Profile(gap,gap_plasmon,wavelength,1,pixel_size = 0.1)
plt.plot(x,np.real(prof),linewidth = 2)


## Following the guided modes

If we work at a specific wavelength, the above methods are enough to know all we need to know about the guided modes.

However, it is often important to know how these modes depend on the incoming light's wavelength. This is what the **follow_guided_modes(...)** function does:

In [ ]:
mat = 1.5
materials = [1., mat**2, 2]

incidence = 0*np.pi/180

structure = np.array([2000, 500])
epaisseurs = np.concatenate(([0],structure,[0]))
stack = [0,1, 2, 0]

wavs = np.linspace(400, 600, 100)
neff_min, neff_max = 1., 2.


chose = Structure(materials, stack, epaisseurs)

indices, follow_modes = follow_guided_modes(chose, wavs, 0, neff_min, neff_max, format="n", plot=True)
print(indices)